### IMPORTS

In [1]:
import pandas as pd
import numpy as np
import pandas_gbq
import datetime
from dateutil.relativedelta import relativedelta
from Roster import *
from gspread_pandas import Spread, conf

### CREDENCIALES

In [2]:
cred = conf.get_config('C:\\Users\\santiago.curat\\Pandas\\PEYA', 'PedidosYa-8b8c4d19f61c.json')

### GOOGLE SHEETS

In [3]:
# Reporte KAM
sheet_id = '1JNywQTVzEQKRwqrJRkpzjiXx5Ly-FldtBMfeSYHuL7w'
wks_name = 'KAM'
sheet = Spread(sheet_id, wks_name, config=cred)
reporte_kams = sheet.sheet_to_df(index=0,header_rows=1)

In [4]:
# Franchises KAM
franchises_kam = [x for x in reporte_kams[reporte_kams.columns[0]].to_list() if x is not '']
# KAMs
kams = [x.upper() for x in reporte_kams[reporte_kams.columns[1]].to_list() if x is not '']

### CONSTANTES

In [5]:
# Fechas
today = datetime.date.today()
if today.day == 1:
    ftm = str(today - relativedelta(days=1))
    itm = str((today - relativedelta(days=1)).replace(day=1))
else:
    itm = str(today.replace(day=1))
    ftm = str(today + relativedelta(months=1) - relativedelta(days=(today + relativedelta(months=1)).day))

### QUERIES

In [6]:
# 31MB
q_partners = '''WITH pm_table AS(
      SELECT pm.restaurant_id AS Id,
             CASE WHEN pm.has_confirmed_orders THEN 'Si' ELSE 'No' END AS Confirmed_Orders_TM,
             CASE WHEN pm.is_online THEN 'Si' ELSE 'No' END AS Online_TM,
             CASE WHEN pm.is_online AND pm.has_confirmed_orders = FALSE THEN 'Si' ELSE 'No' END AS Zombie,
             CASE WHEN pm.is_new_online THEN 'Si' ELSE 'No' END AS New_Online,
             CASE WHEN pm.is_churned THEN 'Si' ELSE 'No' END AS Churn
      FROM `peya-bi-tools-pro.il_core.fact_partners_monthly` AS pm
      WHERE DATE(pm.full_date) BETWEEN DATE('{0}') AND DATE('{1}'))
SELECT p.salesforce_id AS Grid,
       p.partner_id AS Id,
       p.partner_name AS Name,
       IFNULL(p.franchise.franchise_name,'-') AS Franchise,
       p.city.name AS City,
       a.area_name AS Area,
       CASE WHEN p.is_online THEN 'Si' ELSE 'No' END AS Online,
       CASE WHEN p.accepts_and_supports_vouchers THEN 'Si' ELSE 'No' END AS Accepts_Vouchers,
       CASE WHEN p.has_online_payment THEN 'Si' ELSE 'No' END AS Has_PO,
       CASE WHEN p.is_logistic THEN 'Si' ELSE 'No' END AS Logistic,
       CASE WHEN p.is_concept THEN 'Si' ELSE 'No' END AS Concept,
       IFNULL(p.business_type.business_type_name,'-') AS Business,
       IFNULL(p.main_cousine_category_name,'-') AS Main_Cuisine,
       IFNULL(CAST(p.billingInfo.partner_commission AS STRING),'-') AS Commission,
       IFNULL(CAST(p.first_date_online AS STRING),'-') AS First_Date_Online,
       IFNULL(pm.Confirmed_Orders_TM,'No') AS Confirmed_Orders_TM,
       IFNULL(pm.Online_TM,'No') AS Online_TM,
       IFNULL(pm.Zombie,'No') AS Zombie,
       IFNULL(pm.New_Online,'No') AS New_Online,
       IFNULL(pm.Churn,'No') AS Churn,
       IFNULL(sfu.Name,'-') AS Account_Owner,
       IFNULL(p.partner_status,'-') AS BO_Status,
       IFNULL(sfa.account_status,'-') AS SF_Status,
       IFNULL(CAST(p.qty_products_portal AS STRING),'-') AS Qty_Products,
       IFNULL(CAST(p.qty_picts_portal AS STRING),'-') AS Qty_Photos
FROM `peya-bi-tools-pro.il_core.dim_partner` AS p
LEFT JOIN pm_table AS pm ON p.partner_id = pm.Id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_area` AS a ON p.address.area_id = a.area_id
LEFT JOIN `peya-bi-tools-pro.il_salesforce.dim_salesforce_account` AS sfa ON p.salesforce_id = sfa.account_grid
LEFT JOIN `peya-data-origins-pro.cl_salesforce.user` AS sfu ON sfa.owner_id = sfu.Id
WHERE p.country_id = 3
      AND p.salesforce_id IS NOT NULL'''.format(itm,ftm)

In [7]:
# Descargo la data
hue_partners = pd.io.gbq.read_gbq(q_partners, project_id='peya-argentina', dialect='standard')

Downloading: 100%|███████████████████████████████████████████████████████████| 71429/71429 [00:15<00:00, 4625.07rows/s]


In [8]:
# Copio la base
partners = hue_partners.copy()

### TRABAJO

In [9]:
# Coloco el Feudo y Reino
partners['Feudo'] = partners.apply(feudos,axis=1)
partners['Reino'] = partners.apply(reinos,axis=1)

In [10]:
# Marco las KA
partners['Account_Owner'] = partners['Account_Owner'].str.upper()
partners['KAM'] = partners.apply(lambda x: 'Si' if x['Franchise'] in franchises_kam or x['Account_Owner'] in kams else 'No',axis=1)

In [11]:
# Ordeno las columnas
cols = ['Grid','Id','Name','Franchise','City','Area','Feudo','Reino','KAM','Concept','Online','Accepts_Vouchers','Has_PO',
        'Logistic','Business','Main_Cuisine','Commission','First_Date_Online','New_Online','Account_Owner','SF_Status',
        'BO_Status','Churn','Zombie','Confirmed_Orders_TM','Online_TM','Qty_Products','Qty_Photos']
partners = partners[cols]

In [12]:
# Ordeno segun Id
partners.sort_values(by=['Id'],inplace=True)

### CARGA

In [13]:
# Carga Partners PEYA
sheet_id = '1HmAvHYbJJa3JyRTgRJGKXataQB_TvFwNzyo8a6qXd2o'
wks_name = 'Partners PEYA'
sheet = Spread(sheet_id, wks_name, config=cred)
sheet.df_to_sheet(partners, index=False, sheet=wks_name, replace=True)